# Delay to get the first SoH
We want to know how much time we have to wait before getting a SoH for a vin


In [ ]:
from core.s3.s3_utils import S3Service
from core.s3.settings import S3Settings
from core.spark_utils import create_spark_session
import plotly.express as px
from core.stats_utils import *
from core.sql_utils import *
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import re

settings = S3Settings()
from transform.result_week.result_phase_to_result_week import ResultPhaseToResultWeek
spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)

s3 = S3Service()


In [ ]:
oem = "renault"

In [ ]:
phases = s3.read_parquet_df_spark(spark, f"result_phases/result_phases_{oem}.parquet").toPandas()


In [ ]:
with get_connection() as con:
    cursor = con.cursor()
    cursor.execute("""SELECT v.vin as VIN, m.make_name as MAKE, vm.model_name as MODEL, vd.soh as SOH_prod, vd.timestamp, vd.cycles, b.net_capacity  FROM vehicle_data vd 
                   left JOIN vehicle v on v.id = vd.vehicle_id
                    left JOIN vehicle_model vm
                    ON v.vehicle_model_id = vm.id
                    left JOIN make m on m.id = vm.make_id
                    left JOIN battery b on b.id = vm.battery_id""")
    dbeaver_df = cursor.fetchall()
dbeaver_df = pd.DataFrame(dbeaver_df, columns=[desc[0] for desc in cursor.description])

dbeaver_df['soh_prod'] = dbeaver_df['soh_prod'].astype(float)
dbeaver_df['soh_prod'] = np.where(dbeaver_df['soh_prod'] > 1, 1, dbeaver_df['soh_prod'])

In [ ]:
phases["DATE"] = (
            pd.to_datetime(phases["DATETIME_BEGIN"], format="mixed")
            .dt.floor(pd.Timedelta(days=7))
            .dt.tz_localize(None)
            .dt.date.astype("datetime64[ns]")
        )

## 1. Pipeline de production SoH

In [ ]:
a = ResultPhaseToResultWeek("renault", spark= None, has_soh= True, has_soh_oem=False,has_levels = False,)

In [ ]:
def prod_soh(df):
    phases_no_inf = a._replace_inf_soh(df)
    from transform.result_week.config import SOH_FILTER_EVAL_STRINGS
    phases_reame = phases_no_inf.eval(SOH_FILTER_EVAL_STRINGS["renault"])
    phases_freq = a._agg_results_by_update_frequency(phases_reame)

    prod = phases_freq.groupby("VIN", observed=True).apply(a._make_soh_presentable_per_vehicle, include_groups=False).reset_index()
    prod["SOH"] = np.where(prod["SOH"] > 1, 1, prod["SOH"])
    return prod


## 2. Analyse en fonction du temps

In [ ]:

all_results_list = []

unique_vins = phases['VIN'].unique()

for vin in unique_vins:
    print(f"Traitement du VIN: {vin}")
    
    # Filtrer les données pour ce VIN
    vin_test = phases[phases['VIN'] == vin].copy()
    
    if len(vin_test) == 0:
        continue
    
    # Calculer la première date pour ce VIN
    first_date = vin_test['DATE'].min()
    
    # Calculer le nombre de périodes pour ce VIN depuis dbeaver_df
    nbr_periods = dbeaver_df[dbeaver_df['vin'] == vin].shape[0]
    
    if nbr_periods == 0:
        continue
    
    # Créer les périodes
    periods = {i: 7*i for i in range(1, nbr_periods+1)}
    
    # Créer les filtres spécifiques pour ce VIN
    filters_specific_list = []
    prev_days = 0
    
    for period_name, days in periods.items():
        period_df = vin_test[
            (vin_test['DATE'] >= first_date + pd.Timedelta(days=prev_days)) & 
            (vin_test['DATE'] < first_date + pd.Timedelta(days=days))
        ].copy()
        period_df['period'] = period_name
        filters_specific_list.append(period_df)
        prev_days = days
    
    if len(filters_specific_list) == 0:
        continue
    
    filters_specific = pd.concat(filters_specific_list, ignore_index=True)
    
    # Créer les groupes de périodes
    group = {f'{i}_semaines': list(range(1, i+1)) for i in range(1, nbr_periods+1)}
    
    # Calculer les rolling mean SOH pour chaque groupe
    for group_name, group_values in group.items():
        # Filtrer les données pour ce groupe de périodes
        filtered_data = filters_specific[filters_specific['period'].isin(group_values)].copy()
        
        if len(filtered_data) == 0:
            continue
        
        # Calculer prod_soh sur les données filtrées
        prod_result = prod_soh(filtered_data)
        
        # Extraire seulement la colonne SOH et faire un merge sur DATE et VIN
        soh_series = prod_result.set_index(['DATE', 'VIN'])['SOH']
        
        filters_specific = filters_specific.set_index(['DATE', 'VIN'])
        filters_specific[f"{group_name}_rolling_mean_soh"] = soh_series
        filters_specific = filters_specific.reset_index()
    
    # Ajouter les résultats de ce VIN à la liste
    all_results_list.append(filters_specific)

# Concaténer tous les résultats
filters_specific_all_vins = pd.concat(all_results_list, ignore_index=True)


### 2.1. Évaluation des erreurs

In [ ]:
result = filters_specific_all_vins.merge(
    dbeaver_df, 
    left_on=["DATE", "VIN"], 
    right_on=['timestamp', 'vin'], 
    how="left"
)

In [ ]:
result_comparison = result[['VIN', 'DATE', 'period', 'soh_prod'] + [col for col in filters_specific_all_vins.columns if 'rolling_mean_soh' in col]].copy()
semaines_cols = [f"{i}_semaines_rolling_mean_soh" for i in range(1, result_comparison.shape[1] - 3)]
for col in semaines_cols:
    result_comparison[f"{col}_diff_abs"] = abs(result_comparison["soh_prod"] - result_comparison[col])
    result_comparison[f"{col}_diff_pct"] = abs((result_comparison["soh_prod"] - result_comparison[col]) / result_comparison["soh_prod"] * 100)


In [ ]:
result_comparison = result_comparison.drop_duplicates()

In [ ]:
diff_abs_cols = [col for col in result_comparison.columns if col.endswith('_semaines_rolling_mean_soh_diff_abs')]
error_stats = []
for col in diff_abs_cols:
    match = re.search(r'(\d+)_semaines_rolling_mean_soh_diff_abs', col)
    if match:
        n_semaines = int(match.group(1))
        valid_data = result_comparison[col].dropna()
        if len(valid_data) > 0:
            error_stats.append({
                'Nombre_semaines': n_semaines,
                'Erreur_moyenne': valid_data.mean(),
                'Erreur_mediane': valid_data.median(),
                'Erreur_std': valid_data.std(),
                'Erreur_q25': valid_data.quantile(0.25),
                'Erreur_q75': valid_data.quantile(0.75),
                'Erreur_max': valid_data.max(),
                'Nombre_observations': len(valid_data),
                'Nombre_VINs': result_comparison[result_comparison[col].notna()]['VIN'].nunique()
            })

error_df = pd.DataFrame(error_stats).sort_values('Nombre_semaines')
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=error_df['Nombre_semaines'],
    y=error_df['Erreur_q75'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hovertemplate='Q75: %{y:.4f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=error_df['Nombre_semaines'],
    y=error_df['Erreur_q25'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(255, 0, 0, 0.2)',
    fill='tonexty',
    name='Intervalle interquartile (Q25-Q75)',
    hovertemplate='Q25: %{y:.4f}<extra></extra>'
))

# Moyenne ± 2 écarts-types
if 'Erreur_std' in error_df.columns:
    fig.add_trace(go.Scatter(
        x=error_df['Nombre_semaines'],
        y=error_df['Erreur_moyenne'] + 2 * error_df['Erreur_std'],
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hovertemplate='Moyenne + 2σ: %{y:.4f}<extra></extra>'
    ))
    
    fig.add_trace(go.Scatter(
        x=error_df['Nombre_semaines'],
        y=error_df['Erreur_moyenne'] - 2 * error_df['Erreur_std'],
        mode='lines',
        line=dict(width=0),
        fillcolor='rgba(0, 0, 255, 0.15)',
        fill='tonexty',
        name='Moyenne ± 2 écarts-types',
        hovertemplate='Moyenne - 2σ: %{y:.4f}<extra></extra>'
    ))

# Ligne principale avec l'erreur moyenne
fig.add_trace(go.Scatter(
    x=error_df['Nombre_semaines'],
    y=error_df['Erreur_moyenne'],
    mode='lines+markers',
    name='Erreur moyenne',
    line=dict(color='blue', width=3),
    marker=dict(size=10),
    hovertemplate='<b>%{x} semaines</b><br>Erreur moyenne: %{y:.4f}<br>Nombre d\'observations: %{customdata[0]}<br>Nombre de VINs: %{customdata[1]}<extra></extra>',
    customdata=error_df[['Nombre_observations', 'Nombre_VINs']].values
))

# Ligne avec l'erreur maximale
if 'Erreur_max' in error_df.columns:
    fig.add_trace(go.Scatter(
        x=error_df['Nombre_semaines'],
        y=error_df['Erreur_max'],
        mode='markers',
        name='Erreur maximale',
        marker=dict(symbol='triangle-up', size=10, color='orange'),
        hovertemplate='<b>%{x} semaines</b><br>Erreur maximale: %{y:.4f}<extra></extra>'
    ))

fig.update_layout(
    title='Erreur moyenne en fonction du nombre de semaines de suivi (tous les VINs)',
    xaxis_title='Nombre de semaines',
    yaxis_title='Erreur absolue moyenne',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=600
)

fig.show()

# Afficher le tableau récapitulatif
print("\nTableau récapitulatif de l'erreur moyenne par nombre de semaines (tous les VINs):")
display(error_df.round(4))


### 2.3 Identification des outliers

In [ ]:
diff_abs_cols = [col for col in result_comparison.columns if col.endswith('_semaines_rolling_mean_soh_diff_abs')]
seuil_erreur = 0.1
vins_avec_erreur = []

for vin in result_comparison['VIN'].unique():
    vin_data = result_comparison[result_comparison['VIN'] == vin].copy()
    
    erreurs_sup_seuil = []
    erreur_max = 0
    nb_semaines_avec_erreur = 0
    
    for col in diff_abs_cols:
        match = re.search(r'(\d+)_semaines_rolling_mean_soh_diff_abs', col)
        if match:
            n_semaines = int(match.group(1))
            # Récupérer l'erreur maximale pour ce VIN et ce nombre de semaines
            erreur_max_col = vin_data[col].max()
            
            if pd.notna(erreur_max_col) and erreur_max_col > seuil_erreur:
                erreurs_sup_seuil.append({
                    'Nombre_semaines': n_semaines,
                    'Erreur_max': erreur_max_col,
                    'Date': vin_data.loc[vin_data[col].idxmax(), 'DATE'] if vin_data[col].notna().any() else None
                })
                nb_semaines_avec_erreur += 1
                erreur_max = max(erreur_max, erreur_max_col)
    
    # Si ce VIN a au moins une erreur supérieure au seuil
    if erreurs_sup_seuil:
        vins_avec_erreur.append({
            'VIN': vin,
            'Erreur_maximale': erreur_max,
            'Nombre_semaines_avec_erreur': nb_semaines_avec_erreur,
            'Details': erreurs_sup_seuil
        })

# Créer un DataFrame avec les VINs problématiques
if vins_avec_erreur:
    vins_problematiques_df = pd.DataFrame([
        {
            'VIN': item['VIN'],
            'Erreur_maximale': item['Erreur_maximale'],
            'Nombre_semaines_avec_erreur': item['Nombre_semaines_avec_erreur']
        }
        for item in vins_avec_erreur
    ]).sort_values('Erreur_maximale', ascending=False)
    
vins_problematiques_df = vins_problematiques_df.round(4)
vins_problematiques_df.sort_values("Nombre_semaines_avec_erreur", ascending = False)
    


**Note :** Il y a des SoH calculés qui sont des "erreurs".  
Exemple : `VF1AG000765232752` à 5_semaines -> soh_min = 20.59 % et uniquement 2 SoH sur cette période.

### 2.4 Analyse sur un VIN spécifique

In [ ]:
vin = "VF1AG000064548077"
vin_test = phases[phases["VIN"] == vin].copy()
#VF1AG000064548077
#VF1AG000966427889

In [ ]:
first_date = vin_test['DATE'].min()

nbr_periods = dbeaver_df[dbeaver_df['vin'] == vin].shape[0]
periods = {i: 7*i for i in range(1, nbr_periods+1)}

filters_specific_list = []
prev_days = 0
for period_name, days in periods.items():
    print(period_name, days)
    period_df = vin_test[
        (vin_test['DATE'] >= first_date + pd.Timedelta(days=prev_days)) & 
        (vin_test['DATE'] < first_date + pd.Timedelta(days=days))
    ].copy()
    period_df['period'] = period_name
    filters_specific_list.append(period_df)
    prev_days = days
filters_specific_vin = pd.concat(filters_specific_list, ignore_index=True)

In [ ]:
group = {f'{i}_semaines': list(range(1, i+1)) for i in range(1, nbr_periods+1)}


for group_name, group_values in group.items():
    # Filtrer les données pour ce groupe de périodes
    filtered_data = filters_specific_vin[filters_specific_vin['period'].isin(group_values)].copy()
    
    # Calculer prod_soh sur les données filtrées
    prod_result = prod_soh(filtered_data)
    
    # Extraire seulement la colonne SOH et faire un merge sur DATE et VIN
    # Assurez-vous que les colonnes de jointure existent dans les deux DataFrames
    soh_series = prod_result.set_index(['DATE', 'VIN'])['SOH']
    # all_data_sorted["DATE"] = (
    #         pd.to_datetime(all_data_sorted["DATETIME_BEGIN"], format="mixed")
    #         .dt.floor(pd.Timedelta(days=7))
    #         .dt.tz_localize(None)
    #         .dt.date.astype("datetime64[ns]")
    #     )

    filters_specific_vin = filters_specific_vin.set_index(['DATE', 'VIN'])
    filters_specific_vin[f"{group_name}_rolling_mean_soh"] = soh_series.round(3)
    filters_specific_vin = filters_specific_vin.reset_index()


columns_to_show = ['VIN', 'DATE', 'period'] + [col for col in filters_specific_vin.columns if 'rolling_mean_soh' in col]

filters_specific_vin[columns_to_show].drop_duplicates().head(20)

In [ ]:
res = filters_specific_vin.merge(dbeaver_df, left_on = ["DATE", "VIN"], right_on = ['timestamp', 'vin'], how = "left")
res_comparison = res[['VIN', 'DATE', 'period', 'soh_prod'] + [col for col in filters_specific_vin.columns if 'rolling_mean_soh' in col]]
semaines_cols = [f"{i}_semaines_rolling_mean_soh" for i in range(1, res_comparison.shape[1] - 3)]
res_comparison = res_comparison.drop_duplicates()
for col in semaines_cols:
    res_comparison[f"{col}_diff_abs"] = abs(res_comparison["soh_prod"] - res_comparison[col])
    res_comparison[f"{col}_diff_pct"] = abs((res_comparison["soh_prod"] - res_comparison[col]) / res_comparison["soh_prod"] * 100)


In [ ]:
diff_abs_cols = [col for col in res_comparison.columns if col.endswith('_semaines_rolling_mean_soh_diff_abs')]

error_stats = []
for col in diff_abs_cols:
    match = re.search(r'(\d+)_semaines_rolling_mean_soh_diff_abs', col)
    if match:
        n_semaines = int(match.group(1))
        
        # Calculer l'erreur moyenne (en excluant les NaN)
        valid_data = res_comparison[col].dropna()
        if len(valid_data) > 0:
            error_stats.append({
                'Nombre_semaines': n_semaines,
                'Erreur_moyenne': valid_data.mean(),
                'Erreur_mediane': valid_data.median(),
                'Erreur_std': valid_data.std(),
                'Nombre_observations': len(valid_data)
            })


error_df = pd.DataFrame(error_stats).sort_values('Nombre_semaines')


########### raphique
fig = go.Figure()


fig.add_trace(go.Scatter(
    x=error_df['Nombre_semaines'],
    y=error_df['Erreur_moyenne'],
    mode='lines+markers',
    name='Erreur moyenne',
    line=dict(color='blue', width=3),
    marker=dict(size=10),
    hovertemplate='<b>%{x} semaines</b><br>Erreur moyenne: %{y:.4f}<br>Nombre d\'observations: %{customdata}<extra></extra>',
    customdata=error_df['Nombre_observations']
))


if 'Erreur_std' in error_df.columns:
    fig.add_trace(go.Scatter(
        x=error_df['Nombre_semaines'],
        y=error_df['Erreur_moyenne'] + error_df['Erreur_std'],
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hovertemplate='Moyenne + 1σ: %{y:.4f}<extra></extra>'
    ))
    
    fig.add_trace(go.Scatter(
        x=error_df['Nombre_semaines'],
        y=error_df['Erreur_moyenne'] - error_df['Erreur_std'],
        mode='lines',
        line=dict(width=0),
        fillcolor='rgba(0, 0, 255, 0.2)',
        fill='tonexty',
        name='Moyenne ± 1 écart-type',
        hovertemplate='Moyenne - 1σ: %{y:.4f}<extra></extra>'
    ))



fig.update_layout(
    title='Erreur moyenne en fonction du nombre de semaines de suivi',
    xaxis_title='Nombre de semaines',
    yaxis_title='Erreur absolue moyenne',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=600
)

fig.show()

# Afficher le tableau récapitulatif

error_df.round(4)


## 3. Analyse en fonction du nombre de phases

In [ ]:
phases_with_cycles = pd.merge_asof(phases.sort_values("DATETIME_BEGIN"), dbeaver_df.sort_values("timestamp")[['vin', 'cycles', 'timestamp']], left_on = "DATETIME_BEGIN", right_on = "timestamp", left_by = "VIN", right_by = "vin")

In [ ]:
all_results_by_phases_list = []

unique_vins = phases['VIN'].unique()

# Seuils de phases à tester
phase_thresholds = [1, 5, 10, 15, 20, 25, 30, 40, 50, 75, 100]

for vin in unique_vins:
    print(f"Traitement du VIN: {vin}")
    
    # Filtrer les données pour ce VIN et trier par date
    vin_temp = phases[phases['VIN'] == vin].sort_values('DATETIME_BEGIN').copy()
    
    if len(vin_temp) == 0:
        continue
    vin_prod_data = dbeaver_df[dbeaver_df['vin'] == vin].copy()
    if len(vin_prod_data) == 0:
        continue
    for threshold in phase_thresholds:
        if len(vin_temp) < threshold:
            continue
        
        # Prendre les N premières phases
        phases_subset = vin_temp.head(threshold).copy()
        try:
            prod_result = prod_soh(phases_subset)
            
            if len(prod_result) == 0:
                continue
            
            # Pour chaque date où on a un SoH calculé, trouver le soh_prod le plus proche
            for idx, row in prod_result.iterrows():
                soh_calculated = row['SOH']
                date_calculated = row['DATE']
                
                # Trouver le soh_prod le plus proche dans le temps (forward fill)
                closest_prod = vin_prod_data[
                    vin_prod_data['timestamp'] >= date_calculated
                ].sort_values('timestamp') 

                if len(closest_prod) > 0:
                    soh_prod = closest_prod.iloc[0]['soh_prod']
                    
                    all_results_by_phases_list.append({
                        'VIN': vin,
                        'Nombre_phases': threshold,
                        'DATE': date_calculated,
                        'SOH_calculated': soh_calculated,
                        'SOH_prod': soh_prod,
                        'Erreur_abs': abs(soh_prod - soh_calculated),
                        'Erreur_pct': abs((soh_prod - soh_calculated) / soh_prod * 100) if soh_prod > 0 else None
                    })
        except Exception as e:
            print(f"Erreur pour VIN {vin} avec {threshold} phases: {e}")
            continue

# Créer le DataFrame avec tous les résultats
results_by_phases_df = pd.DataFrame(all_results_by_phases_list)


In [ ]:
# Calculer les statistiques d'erreur par nombre de phases
error_stats_by_phases = []

for threshold in phase_thresholds:
    threshold_data = results_by_phases_df[results_by_phases_df['Nombre_phases'] == threshold]
    
    if len(threshold_data) == 0:
        continue
    
    valid_errors = threshold_data['Erreur_abs'].dropna()
    
    if len(valid_errors) > 0:
        error_stats_by_phases.append({
            'Nombre_phases': threshold,
            'Erreur_moyenne': valid_errors.mean(),
            'Erreur_mediane': valid_errors.median(),
            'Erreur_std': valid_errors.std(),
            'Erreur_q25': valid_errors.quantile(0.25),
            'Erreur_q75': valid_errors.quantile(0.75),
            'Erreur_max': valid_errors.max(),
            'Nombre_observations': len(valid_errors),
            'Nombre_VINs': threshold_data['VIN'].nunique()
        })

error_by_phases_df = pd.DataFrame(error_stats_by_phases).sort_values('Nombre_phases')

display(error_by_phases_df.round(4))

In [ ]:
# Graphique de l'erreur en fonction du nombre de phases
fig = go.Figure()

# Intervalle interquartile (Q25-Q75)
fig.add_trace(go.Scatter(
    x=error_by_phases_df['Nombre_phases'],
    y=error_by_phases_df['Erreur_q75'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hovertemplate='Q75: %{y:.4f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=error_by_phases_df['Nombre_phases'],
    y=error_by_phases_df['Erreur_q25'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(255, 0, 0, 0.2)',
    fill='tonexty',
    name='Intervalle interquartile (Q25-Q75)',
    hovertemplate='Q25: %{y:.4f}<extra></extra>'
))

# Moyenne ± 2 écarts-types
if 'Erreur_std' in error_by_phases_df.columns:
    fig.add_trace(go.Scatter(
        x=error_by_phases_df['Nombre_phases'],
        y=error_by_phases_df['Erreur_moyenne'] + 2 * error_by_phases_df['Erreur_std'],
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hovertemplate='Moyenne + 2σ: %{y:.4f}<extra></extra>'
    ))
    
    fig.add_trace(go.Scatter(
        x=error_by_phases_df['Nombre_phases'],
        y=error_by_phases_df['Erreur_moyenne'] - 2 * error_by_phases_df['Erreur_std'],
        mode='lines',
        line=dict(width=0),
        fillcolor='rgba(0, 0, 255, 0.15)',
        fill='tonexty',
        name='Moyenne ± 2 écarts-types',
        hovertemplate='Moyenne - 2σ: %{y:.4f}<extra></extra>'
    ))

# Ligne principale avec l'erreur moyenne
fig.add_trace(go.Scatter(
    x=error_by_phases_df['Nombre_phases'],
    y=error_by_phases_df['Erreur_moyenne'],
    mode='lines+markers',
    name='Erreur moyenne',
    line=dict(color='blue', width=3),
    marker=dict(size=10),
    hovertemplate='<b>%{x} phases</b><br>Erreur moyenne: %{y:.4f}<br>Nombre d\'observations: %{customdata[0]}<br>Nombre de VINs: %{customdata[1]}<extra></extra>',
    customdata=error_by_phases_df[['Nombre_observations', 'Nombre_VINs']].values
))

# Ligne avec l'erreur maximale
if 'Erreur_max' in error_by_phases_df.columns:
    fig.add_trace(go.Scatter(
        x=error_by_phases_df['Nombre_phases'],
        y=error_by_phases_df['Erreur_max'],
        mode='markers',
        name='Erreur maximale',
        marker=dict(symbol='triangle-up', size=10, color='orange'),
        hovertemplate='<b>%{x} phases</b><br>Erreur maximale: %{y:.4f}<extra></extra>'
    ))

fig.update_layout(
    title='Erreur moyenne en fonction du nombre de phases avec SoH calculé (tous les VINs)',
    xaxis_title='Nombre de phases',
    yaxis_title='Erreur absolue moyenne',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=600
)

fig.show()

# Afficher le tableau récapitulatif
print("\nTableau récapitulatif de l'erreur moyenne par nombre de phases (tous les VINs):")
display(error_by_phases_df.round(4))


## 4. Analyse en fonction du nombre de cycles

In [ ]:
phases['odometer_start_following'] =  phases.groupby('VIN')['ODOMETER_FIRST'].transform('first')

In [ ]:
all_results_by_cycles_list = []

unique_vins = phases['VIN'].unique()

# Seuils de cycles à tester
cycle_thresholds = [5, 10, 15, 20, 25, 30, 40, 50, 75, 100]

for vin in unique_vins:
    print(f"Traitement du VIN: {vin}")
    
    # Filtrer les données pour ce VIN et trier par date
    vin_temp = phases[phases['VIN'] == vin].sort_values('DATETIME_BEGIN').copy()
    
    if len(vin_temp) == 0:
        continue
    
    vin_prod_data = dbeaver_df[dbeaver_df['vin'] == vin].copy()
    if len(vin_prod_data) == 0:
        continue
    
    # Vérifier que les colonnes nécessaires existent
    if 'ODOMETER_LAST' not in vin_temp.columns or 'RANGE' not in vin_temp.columns:
        continue
    
    if 'odometer_start_following' not in vin_temp.columns:
        continue
    
    # Calculer les cycles pour chaque phase en utilisant la différence entre ODOMETER_LAST et odometer_start_following
    # Calculer la distance parcourue depuis le début du suivi
    vin_temp['DISTANCE_FOLLOWING'] = vin_temp['ODOMETER_LAST'] - vin_temp['odometer_start_following']
    
    # Calculer les cycles estimés pour chaque phase
    # Utiliser la même formule que estimate_cycles mais avec la distance parcourue depuis le début du suivi
    if 'SOH' in vin_temp.columns:
        vin_temp['ESTIMATED_CYCLES'] = vin_temp.apply(
            lambda row: round(row['DISTANCE_FOLLOWING'] / (row['RANGE'] * (row['SOH'] + 1) / 2)) if pd.notna(row['DISTANCE_FOLLOWING']) and pd.notna(row['RANGE']) and pd.notna(row['SOH']) and row['RANGE'] > 0 else np.nan,
            axis=1
        )
    else:
        vin_temp['ESTIMATED_CYCLES'] = vin_temp.apply(
            lambda row: round(row['DISTANCE_FOLLOWING'] / row['RANGE']) if pd.notna(row['DISTANCE_FOLLOWING']) and pd.notna(row['RANGE']) and row['RANGE'] > 0 else np.nan,
            axis=1
        )
    
    for threshold in cycle_thresholds:
        # Prendre toutes les phases où les cycles estimés sont <= au seuil
        # (les cycles estimés augmentent avec l'odomètre, donc cela prend toutes les phases jusqu'à ce seuil)
        phases_subset = vin_temp[vin_temp['ESTIMATED_CYCLES'] <= threshold].copy()
        
        if len(phases_subset) == 0:
            continue
        
        try:
            prod_result = prod_soh(phases_subset)
            
            if len(prod_result) == 0:
                continue
            
            # Pour chaque date où on a un SoH calculé, trouver le soh_prod le plus proche
            for idx, row in prod_result.iterrows():
                soh_calculated = row['SOH']
                date_calculated = row['DATE']
                
                # Trouver le soh_prod le plus proche dans le temps (forward fill)
                closest_prod = vin_prod_data[
                    vin_prod_data['timestamp'] >= date_calculated
                ].sort_values('timestamp')
                
                if len(closest_prod) > 0:
                    soh_prod = closest_prod.iloc[0]['soh_prod']
                    
                    all_results_by_cycles_list.append({
                        'VIN': vin,
                        'Nombre_cycles': threshold,
                        'DATE': date_calculated,
                        'SOH_calculated': soh_calculated,
                        'SOH_prod': soh_prod,
                        'Erreur_abs': abs(soh_prod - soh_calculated),
                        'Erreur_pct': abs((soh_prod - soh_calculated) / soh_prod * 100) if soh_prod > 0 else None
                    })
        except Exception as e:
            print(f"Erreur pour VIN {vin} avec {threshold} cycles: {e}")
            continue

results_by_cycles_df = pd.DataFrame(all_results_by_cycles_list)


In [ ]:
# Calculer les statistiques d'erreur par nombre de cycles
error_stats_by_cycles = []

for threshold in cycle_thresholds:
    threshold_data = results_by_cycles_df[results_by_cycles_df['Nombre_cycles'] == threshold]
    
    if len(threshold_data) == 0:
        continue
    
    valid_errors = threshold_data['Erreur_abs'].dropna()
    
    if len(valid_errors) > 0:
        error_stats_by_cycles.append({
            'Nombre_cycles': threshold,
            'Erreur_moyenne': valid_errors.mean(),
            'Erreur_mediane': valid_errors.median(),
            'Erreur_std': valid_errors.std(),
            'Erreur_q25': valid_errors.quantile(0.25),
            'Erreur_q75': valid_errors.quantile(0.75),
            'Erreur_max': valid_errors.max(),
            'Nombre_observations': len(valid_errors),
            'Nombre_VINs': threshold_data['VIN'].nunique()
        })

error_by_cycles_df = pd.DataFrame(error_stats_by_cycles).sort_values('Nombre_cycles')

display(error_by_cycles_df.round(4))


In [ ]:
# Graphique de l'erreur en fonction du nombre de cycles
fig = go.Figure()

# Intervalle interquartile (Q25-Q75)
fig.add_trace(go.Scatter(
    x=error_by_cycles_df['Nombre_cycles'],
    y=error_by_cycles_df['Erreur_q75'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hovertemplate='Q75: %{y:.4f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=error_by_cycles_df['Nombre_cycles'],
    y=error_by_cycles_df['Erreur_q25'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(255, 0, 0, 0.2)',
    fill='tonexty',
    name='Intervalle interquartile (Q25-Q75)',
    hovertemplate='Q25: %{y:.4f}<extra></extra>'
))

# Moyenne ± 2 écarts-types
if 'Erreur_std' in error_by_cycles_df.columns:
    fig.add_trace(go.Scatter(
        x=error_by_cycles_df['Nombre_cycles'],
        y=error_by_cycles_df['Erreur_moyenne'] + 2 * error_by_cycles_df['Erreur_std'],
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hovertemplate='Moyenne + 2σ: %{y:.4f}<extra></extra>'
    ))
    
    fig.add_trace(go.Scatter(
        x=error_by_cycles_df['Nombre_cycles'],
        y=error_by_cycles_df['Erreur_moyenne'] - 2 * error_by_cycles_df['Erreur_std'],
        mode='lines',
        line=dict(width=0),
        fillcolor='rgba(0, 0, 255, 0.15)',
        fill='tonexty',
        name='Moyenne ± 2 écarts-types',
        hovertemplate='Moyenne - 2σ: %{y:.4f}<extra></extra>'
    ))

# Ligne principale avec l'erreur moyenne
fig.add_trace(go.Scatter(
    x=error_by_cycles_df['Nombre_cycles'],
    y=error_by_cycles_df['Erreur_moyenne'],
    mode='lines+markers',
    name='Erreur moyenne',
    line=dict(color='blue', width=3),
    marker=dict(size=10),
    hovertemplate='<b>%{x} cycles</b><br>Erreur moyenne: %{y:.4f}<br>Nombre d\'observations: %{customdata[0]}<br>Nombre de VINs: %{customdata[1]}<extra></extra>',
    customdata=error_by_cycles_df[['Nombre_observations', 'Nombre_VINs']].values
))

# Ligne avec l'erreur maximale
if 'Erreur_max' in error_by_cycles_df.columns:
    fig.add_trace(go.Scatter(
        x=error_by_cycles_df['Nombre_cycles'],
        y=error_by_cycles_df['Erreur_max'],
        mode='markers',
        name='Erreur maximale',
        marker=dict(symbol='triangle-up', size=10, color='orange'),
        hovertemplate='<b>%{x} cycles</b><br>Erreur maximale: %{y:.4f}<extra></extra>'
    ))

fig.update_layout(
    title='Erreur moyenne en fonction du nombre de cycles avec SoH calculé (tous les VINs)',
    xaxis_title='Nombre de cycles',
    yaxis_title='Erreur absolue moyenne',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=600
)

fig.show()

# Afficher le tableau récapitulatif
print("\nTableau récapitulatif de l'erreur moyenne par nombre de cycles (tous les VINs):")
display(error_by_cycles_df.round(4))


### Analyse en fonction du nombre de points

Penser à commenter l'aggregation dans transform.processed_phases.raw_ts_to_processed_phases avant de lancer cette partie partie 

In [ ]:
from transform.processed_phases.providers.renault import RenaultRawTsToProcessedPhases

In [ ]:
renault = RenaultRawTsToProcessedPhases(spark=spark)

In [ ]:
raw_tss_pd = renault.run().toPandas()
raw_tss_pd.columns


In [ ]:
raw_tss_pd.rename(columns={'soh': 'SOH'}, inplace=True)
raw_tss_pd['consumption'] = 0

In [ ]:
raw_tss_pd['charging_rate'] = raw_tss_pd['charging_rate'].astype(float)

In [ ]:
def aggregate_stats(phase_df):
    """
    Version pandas de la fonction aggregate_stats.
    Agrège les statistiques par phase en utilisant pandas au lieu de PySpark.
    """
    # Définir les colonnes de groupement
    groupby_cols = [
        "VIN",
        "PHASE_INDEX",
        "DATETIME_BEGIN",
        "DATETIME_END",
        "PHASE_STATUS",
        "SOC_FIRST",
        "SOC_LAST",
        "SOC_DIFF",
        "NO_SOC_DATAPOINT",
        "IS_USABLE_PHASE",
    ]
    
    # Construire le dictionnaire d'agrégation avec pd.NamedAgg
    agg_dict = {}
    
    # Colonnes avec first (ignore les nulls par défaut en pandas)
    if "make" in phase_df.columns:
        agg_dict["MAKE"] = pd.NamedAgg(column="make", aggfunc="first")
    if "model" in phase_df.columns:
        agg_dict["MODEL"] = pd.NamedAgg(column="model", aggfunc="first")
    if "version" in phase_df.columns:
        agg_dict["VERSION"] = pd.NamedAgg(column="version", aggfunc="first")
    if "net_capacity" in phase_df.columns:
        agg_dict["BATTERY_NET_CAPACITY"] = pd.NamedAgg(column="net_capacity", aggfunc="first")
    if "range" in phase_df.columns:
        agg_dict["RANGE"] = pd.NamedAgg(column="range", aggfunc="first")
    
    # Odometer: first et last
    if "odometer" in phase_df.columns:
        agg_dict["ODOMETER_FIRST"] = pd.NamedAgg(column="odometer", aggfunc="first")
        agg_dict["ODOMETER_LAST"] = pd.NamedAgg(column="odometer", aggfunc="last")
    
    # SOH: percentile_approx(soh, 0.5) -> quantile(0.5) ou median
    if "soh" in phase_df.columns:
        agg_dict["SOH"] = pd.NamedAgg(column="soh", aggfunc=lambda x: x.quantile(0.5))
    
    # Charging rate: mean
    if "charging_rate" in phase_df.columns:
        agg_dict["CHARGING_RATE"] = pd.NamedAgg(column="charging_rate", aggfunc="mean")
    
    # Consumption: mean (si présent)
    if "consumption" in phase_df.columns:
        agg_dict["CONSUMPTION"] = pd.NamedAgg(column="consumption", aggfunc="mean")
    
    agg_dict["SOH"] = pd.NamedAgg(column="SOH", aggfunc="median")
    
    # Grouper et agréger
    df_aggregated = phase_df.groupby(groupby_cols, observed=True).agg(**agg_dict).reset_index()
    
    return df_aggregated

In [ ]:
# Analyse en fonction du nombre de points de données raw_tss
all_results_by_points_list = []

unique_vins = raw_tss_pd['VIN'].unique()

# Seuils de points à tester
point_thresholds = [1, 10, 25, 50, 100, 200, 500]

for vin in unique_vins:
    
    # Filtrer les données pour ce VIN et trier par date
    vin_temp = raw_tss_pd[raw_tss_pd['VIN'] == vin].sort_values('date').copy()

    
    vin_prod_data = dbeaver_df[dbeaver_df['vin'] == vin].copy()
    
    
    for threshold in point_thresholds:
        print(threshold)
        
        # Prendre les N premiers points
        points_subset = vin_temp.head(threshold).copy()
        
        # Filtrer les points avec un soh valide (non null et entre 0 et 1)
        valid_soh_points = points_subset[
            (points_subset['SOH'].notna()) & 
            (points_subset['SOH'] > 0) & 
            (points_subset['SOH'] <= 1)
        ]
        
        if len(valid_soh_points) == 0:
            continue
        
        phase_soh = aggregate_stats(valid_soh_points)
        phase_soh['ODOMETER_DIFF'] = phase_soh['ODOMETER_LAST'] - phase_soh['ODOMETER_FIRST']
        phase_prod = prod_soh(phase_soh)
        # Utiliser la date du dernier point pour la comparaison
        last_date = points_subset['date'].max()
        
        # Trouver le soh_prod le plus proche dans le temps (forward fill)
        closest_prod = vin_prod_data[
            vin_prod_data['timestamp'] >= last_date
        ].sort_values('timestamp')
        
        if len(closest_prod) > 0:
            soh_prod = closest_prod.iloc[0]['soh_prod']
            
            # Calculer les erreurs pour la moyenne et la médiane
            all_results_by_points_list.append({
                'VIN': vin,
                'Nombre_points': threshold,
                'DATE': last_date,
                'SOH_calculated_mean': phase_prod['SOH'].values[-1],
                'SOH_prod': soh_prod,
                'Erreur_abs_mean': abs(soh_prod - phase_prod['SOH'].values[-1]),
                'Erreur_pct_mean': abs((soh_prod - phase_prod['SOH'].values[-1]) / soh_prod * 100) if soh_prod > 0 else None,
                'Nombre_points_valides': len(phase_prod['SOH'])
            })

# Créer le DataFrame avec tous les résultats
results_by_points_df = pd.DataFrame(all_results_by_points_list)

In [ ]:
# Calculer les statistiques d'erreur par nombre de points (en utilisant la moyenne)
error_stats_by_points = []

for threshold in point_thresholds:
    threshold_data = results_by_points_df[results_by_points_df['Nombre_points'] == threshold]
    
    if len(threshold_data) == 0:
        continue
    
    valid_errors = threshold_data['Erreur_abs_mean'].dropna()
    
    if len(valid_errors) > 0:
        error_stats_by_points.append({
            'Nombre_points': threshold,
            'Erreur_moyenne': valid_errors.mean(),
            'Erreur_mediane': valid_errors.median(),
            'Erreur_std': valid_errors.std(),
            'Erreur_q25': valid_errors.quantile(0.25),
            'Erreur_q75': valid_errors.quantile(0.75),
            'Erreur_max': valid_errors.max(),
            'Nombre_observations': len(valid_errors),
            'Nombre_VINs': threshold_data['VIN'].nunique()
        })

error_by_points_df = pd.DataFrame(error_stats_by_points).sort_values('Nombre_points')

display(error_by_points_df.round(4))

In [ ]:
# Graphique de l'erreur en fonction du nombre de points
fig = go.Figure()

# Intervalle interquartile (Q25-Q75)
fig.add_trace(go.Scatter(
    x=error_by_points_df['Nombre_points'],
    y=error_by_points_df['Erreur_q75'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hovertemplate='Q75: %{y:.4f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=error_by_points_df['Nombre_points'],
    y=error_by_points_df['Erreur_q25'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(255, 0, 0, 0.2)',
    fill='tonexty',
    name='Intervalle interquartile (Q25-Q75)',
    hovertemplate='Q25: %{y:.4f}<extra></extra>'
))

# Moyenne ± 2 écarts-types
if 'Erreur_std' in error_by_points_df.columns:
    fig.add_trace(go.Scatter(
        x=error_by_points_df['Nombre_points'],
        y=error_by_points_df['Erreur_moyenne'] + 2 * error_by_points_df['Erreur_std'],
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hovertemplate='Moyenne + 2σ: %{y:.4f}<extra></extra>'
    ))
    
    fig.add_trace(go.Scatter(
        x=error_by_points_df['Nombre_points'],
        y=error_by_points_df['Erreur_moyenne'] - 2 * error_by_points_df['Erreur_std'],
        mode='lines',
        line=dict(width=0),
        fillcolor='rgba(0, 0, 255, 0.15)',
        fill='tonexty',
        name='Moyenne ± 2 écarts-types',
        hovertemplate='Moyenne - 2σ: %{y:.4f}<extra></extra>'
    ))

# Ligne principale avec l'erreur moyenne
fig.add_trace(go.Scatter(
    x=error_by_points_df['Nombre_points'],
    y=error_by_points_df['Erreur_moyenne'],
    mode='lines+markers',
    name='Erreur moyenne',
    line=dict(color='blue', width=3),
    marker=dict(size=10),
    hovertemplate='<b>%{x} points</b><br>Erreur moyenne: %{y:.4f}<br>Nombre d\'observations: %{customdata[0]}<br>Nombre de VINs: %{customdata[1]}<extra></extra>',
    customdata=error_by_points_df[['Nombre_observations', 'Nombre_VINs']].values
))

# Ligne avec l'erreur maximale
if 'Erreur_max' in error_by_points_df.columns:
    fig.add_trace(go.Scatter(
        x=error_by_points_df['Nombre_points'],
        y=error_by_points_df['Erreur_max'],
        mode='markers',
        name='Erreur maximale',
        marker=dict(symbol='triangle-up', size=10, color='orange'),
        hovertemplate='<b>%{x} points</b><br>Erreur maximale: %{y:.4f}<extra></extra>'
    ))

fig.update_layout(
    title='Erreur moyenne en fonction du nombre de points de données raw_tss (tous les VINs)',
    xaxis_title='Nombre de points',
    yaxis_title='Erreur absolue moyenne',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=600
)

fig.show()

# Afficher le tableau récapitulatif
print("\nTableau récapitulatif de l'erreur moyenne par nombre de points (tous les VINs):")
display(error_by_points_df.round(4))